# Atividade 1
## Item 2 - Problema MAX-QBF - Sem linearização

In [4]:
import gurobipy as gp
from gurobipy import GRB

In [8]:
def solve_max_qbf(n, a_coeffs):
    """
    Resolve o problema Unconstrained Binary Quadratic Programming (MAX-QBF).

    Args:
        n (int): Número de variáveis binárias.
        a_coeffs (list): Lista linearizada dos coeficientes da matriz A
                         em formato triangular superior.
    """
    # 1. Reconstruir a matriz A (assumindo triangular superior)
    A = [[0.0] * n for _ in range(n)]
    k = 0
    for i in range(n):
        for j in range(i, n):
            A[i][j] = a_coeffs[k]
            k += 1

    # Imprimir a matriz A reconstruída para verificação
    print("Matriz A de Coeficientes:")
    for row in A:
        print(row)
    print("-" * 30)

    # 2. Criar um novo modelo Gurobi
    model = gp.Model("MAX_QBF_Problem")

    # 3. Adicionar as variáveis binárias
    # x[i] representa a i-ésima variável binária (indexado de 0 a n-1)
    x = model.addVars(n, vtype=GRB.BINARY, name="x")

    # 4. Definir a função objetivo (expressão quadrática)
    # A Gurobi permite a criação de funções objetivo quadráticas diretamente.
    # O loop duplo itera sobre todos os pares (i, j) para construir a soma.
    # Para i=j, A[i][i]*x[i]*x[i] se torna A[i][i]*x[i] (termo linear).
    # Para i<j, A[i][j]*x[i]*x[j] é um termo quadrático.
    # Como a matriz é triangular superior, A[j][i] (para j<i) seria zero.
    
    # Construção da expressão quadrática
    obj = gp.quicksum(A[i][j] * x[i] * x[j] for i in range(n) for j in range(n))
    
    # 5. Definir o sentido da otimização (Maximizar)
    model.setObjective(obj, GRB.MAXIMIZE)

    # Configurar limite de tempo (10 minutos conforme atividade)
    model.Params.TimeLimit = 600 # segundos

    # 6. Otimizar o modelo
    print("Iniciando otimização do MAX-QBF...")
    model.optimize()

    # 7. Obter e exibir os resultados
    print("\n" + "=" * 40)
    print("Resultados da Otimização:")
    print("=" * 40)

    if model.status == GRB.OPTIMAL:
        print(f"Status: Ótimo (Optimal)")
        print(f"Valor da Solução Ótima (Z): {model.ObjVal:.4f}")
        print("Valores das Variáveis:")
        for i in range(n):
            print(f"  x[{i+1}] = {int(x[i].X)}") # Apresentando como x[5] a x[n]
    elif model.status == GRB.TIME_LIMIT:
        print(f"Status: Limite de tempo atingido (Time Limit)")
        print(f"Melhor Solução Encontrada (Z): {model.ObjVal:.4f}")
        print(f"Gap de Otimalidade: {model.MIPGap * 100:.2f}%")
        print("Valores das Variáveis na melhor solução:")
        for i in range(n):
            print(f"  x[{i+1}] = {int(x[i].X)}")
    elif model.status == GRB.INFEASIBLE:
        print("Status: Inviável (Infeasible)")
    else:
        print(f"Status da Otimização: {model.status}")

In [9]:
n_example = 5
a_coeffs_example = [
     3, 1, 2, 0, 3,  # a11 a12 a13 a14 a15
        1, 2, 1, 1,  #     a22 a23 a24 a25
           2, 2, 4,  #         a33 a34 a35
              0, 5,  #             a44 a45
                 3   #                 a55
]

In [10]:
# Rodar a função com os dados de exemplo
solve_max_qbf(n_example, a_coeffs_example)

Matriz A de Coeficientes:
[3, 1, 2, 0, 3]
[0.0, 1, 2, 1, 1]
[0.0, 0.0, 2, 2, 4]
[0.0, 0.0, 0.0, 0, 5]
[0.0, 0.0, 0.0, 0.0, 3]
------------------------------
Set parameter TimeLimit to value 600
Iniciando otimização do MAX-QBF...
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  600

Optimize a model with 0 rows, 5 columns and 0 nonzeros
Model fingerprint: 0xf26c5b89
Model has 13 quadratic objective terms
Variable types: 0 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 30.0000000
Presolve removed 0 row

In [20]:
# Exemplo com n=3 para ilustrar mais claramente a matriz A e o objetivo
print("\n" + "=" * 40)
print("Exemplo Adicional: n=3")
print("=" * 40)
n_small = 3

a_coeffs_small = [-5, -6, 7, 8, 9, -10] 
solve_max_qbf(n_small, a_coeffs_small)


Exemplo Adicional: n=3
Matriz A de Coeficientes:
[-5, -6, 7]
[0.0, 8, 9]
[0.0, 0.0, -10]
------------------------------
Set parameter TimeLimit to value 600
Iniciando otimização do MAX-QBF...
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  600

Optimize a model with 0 rows, 3 columns and 0 nonzeros
Model fingerprint: 0xb0301e19
Model has 6 quadratic objective terms
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [1e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 3.0000000
Found heuristic solution: objective 8.0000000
Presolve remov